# Чат-бот в Telegram

**Задача**: на базе библиотеки python-telegram-bot и библиотеки вопросов-ответов mail.ru реализовать чат-бота в телеграм, который будет находить максимально подходящие ответы с использованием библиотек NLP ML.  

Ссылка на чат-бот: http://t.me/InPROBot  

---

## Библиотеки, настройки и скрипты

In [1]:
# Задаем кодировку
# -- coding: utf-8 -

In [2]:
import numpy as np
import pandas as pd
import os

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText
import pickle
from tqdm import tqdm_notebook
import string
import annoy

import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import dialogflow

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

/home/vs/.local/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
# Настройки бота
token = '627042784:AAF-g4PmAtX6_HMHrpqhbz-C4E2nT5AXuAo'
updater = Updater(token, use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

In [4]:
# Включаем логгирование
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger()

## Предобработка ответов

In [6]:
assert True
question = None
written = False

with open('./data/prepared_answers.txt', 'w', encoding='utf-8') as fout:
    with open('./data/Otvety.txt', 'r', encoding='utf-8') as fin:
        for line in tqdm_notebook(fin):
            if line.startswith('---'):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace('\t', ' ').strip() + '\t' + line.replace('\t', ' '))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

<ipython-input-6-8e661ae33903>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [7]:
def preprocess_txt(line):
    spls = ''.join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != '']
    return spls

In [8]:
morpher = MorphAnalyzer()
sw = set(get_stop_words('ru'))
exclude = set(string.punctuation)

2021-08-30 20:11:43,722 - pymorphy2.opencorpora_dict.wrapper - INFO - Loading dictionaries from /home/vs/.local/lib/python3.8/site-packages/pymorphy2_dicts_ru/data
2021-08-30 20:11:43,745 - pymorphy2.opencorpora_dict.wrapper - INFO - format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


## Предобработка моделей

In [9]:
assert True
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words('ru'))
exclude = set(string.punctuation)
c = 0

with open('./data/Otvety.txt', 'r', encoding='utf-8') as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

2021-08-30 20:11:43,782 - pymorphy2.opencorpora_dict.wrapper - INFO - Loading dictionaries from /home/vs/.local/lib/python3.8/site-packages/pymorphy2_dicts_ru/data
2021-08-30 20:11:43,803 - pymorphy2.opencorpora_dict.wrapper - INFO - format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345
<ipython-input-9-1863ce59d9f2>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [10]:
sentences = [i for i in sentences if len(i) > 2]
print('start train fasttext')
modelFT = FastText(sentences=sentences, min_count=1, window=5, workers=8)
#modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5, workers=8)
modelFT.save('ft_model')

2021-08-30 20:31:54,939 - gensim.models.word2vec - INFO - collecting all words and their counts
2021-08-30 20:31:54,940 - gensim.models.word2vec - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-08-30 20:31:54,966 - gensim.models.word2vec - INFO - PROGRESS: at sentence #10000, processed 173060 words, keeping 41540 word types
2021-08-30 20:31:54,997 - gensim.models.word2vec - INFO - PROGRESS: at sentence #20000, processed 353487 words, keeping 68403 word types
2021-08-30 20:31:55,029 - gensim.models.word2vec - INFO - PROGRESS: at sentence #30000, processed 527016 words, keeping 91256 word types
2021-08-30 20:31:55,060 - gensim.models.word2vec - INFO - PROGRESS: at sentence #40000, processed 702279 words, keeping 113049 word types
2021-08-30 20:31:55,092 - gensim.models.word2vec - INFO - PROGRESS: at sentence #50000, processed 885642 words, keeping 134773 word types
2021-08-30 20:31:55,129 - gensim.models.word2vec - INFO - PROGRESS: at sentence #60000, proce

start train fasttext


2021-08-30 20:31:55,168 - gensim.models.word2vec - INFO - PROGRESS: at sentence #70000, processed 1254880 words, keeping 174973 word types
2021-08-30 20:31:55,204 - gensim.models.word2vec - INFO - PROGRESS: at sentence #80000, processed 1426135 words, keeping 192686 word types
2021-08-30 20:31:55,243 - gensim.models.word2vec - INFO - PROGRESS: at sentence #90000, processed 1614425 words, keeping 211276 word types
2021-08-30 20:31:55,278 - gensim.models.word2vec - INFO - PROGRESS: at sentence #100000, processed 1795447 words, keeping 228824 word types
2021-08-30 20:31:55,314 - gensim.models.word2vec - INFO - PROGRESS: at sentence #110000, processed 1979304 words, keeping 245492 word types
2021-08-30 20:31:55,348 - gensim.models.word2vec - INFO - PROGRESS: at sentence #120000, processed 2161638 words, keeping 262687 word types
2021-08-30 20:31:55,385 - gensim.models.word2vec - INFO - PROGRESS: at sentence #130000, processed 2356266 words, keeping 279849 word types
2021-08-30 20:31:55,424

2021-08-30 20:32:47,609 - gensim.models.word2vec - INFO - worker thread finished; awaiting finish of 0 more threads
2021-08-30 20:32:47,610 - gensim.models.word2vec - INFO - EPOCH - 1 : training on 5981241 raw words (5918009 effective words) took 9.2s, 639951 effective words/s
2021-08-30 20:32:48,657 - gensim.models.word2vec - INFO - EPOCH 2 - PROGRESS: at 11.52% examples, 619110 words/s, in_qsize 16, out_qsize 0
2021-08-30 20:32:49,683 - gensim.models.word2vec - INFO - EPOCH 2 - PROGRESS: at 23.83% examples, 653747 words/s, in_qsize 16, out_qsize 1
2021-08-30 20:32:50,712 - gensim.models.word2vec - INFO - EPOCH 2 - PROGRESS: at 35.75% examples, 660810 words/s, in_qsize 14, out_qsize 1
2021-08-30 20:32:51,742 - gensim.models.word2vec - INFO - EPOCH 2 - PROGRESS: at 47.03% examples, 659530 words/s, in_qsize 16, out_qsize 1
2021-08-30 20:32:52,754 - gensim.models.word2vec - INFO - EPOCH 2 - PROGRESS: at 58.08% examples, 666465 words/s, in_qsize 16, out_qsize 0
2021-08-30 20:32:53,795 - g

2021-08-30 20:33:22,916 - gensim.models.word2vec - INFO - worker thread finished; awaiting finish of 5 more threads
2021-08-30 20:33:22,963 - gensim.models.word2vec - INFO - worker thread finished; awaiting finish of 4 more threads
2021-08-30 20:33:22,964 - gensim.models.word2vec - INFO - worker thread finished; awaiting finish of 3 more threads
2021-08-30 20:33:22,969 - gensim.models.word2vec - INFO - worker thread finished; awaiting finish of 2 more threads
2021-08-30 20:33:22,974 - gensim.models.word2vec - INFO - worker thread finished; awaiting finish of 1 more threads
2021-08-30 20:33:22,977 - gensim.models.word2vec - INFO - worker thread finished; awaiting finish of 0 more threads
2021-08-30 20:33:22,977 - gensim.models.word2vec - INFO - EPOCH - 5 : training on 5981241 raw words (5918075 effective words) took 8.9s, 663370 effective words/s
2021-08-30 20:33:22,978 - gensim.utils - INFO - FastText lifecycle event {'msg': 'training on 29906205 raw words (29589607 effective words) to

In [11]:
#modelFT = FastText.load('ft_model')
ft_index = annoy.AnnoyIndex(100, 'angular')

index_map = {}
counter = 0

with open('./data/prepared_answers.txt', 'r') as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split('\t')
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

ft_index.build(10)
ft_index.save('./data/speaker.ann')

<ipython-input-11-38f4cbcfeae7>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


0it [00:00, ?it/s]

True

In [13]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('./data/speaker.ann') 

True

In [14]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[3527, 28954]

In [15]:
shop_data = pd.read_csv('./data/ProductsDataset.csv')
shop_data['text'] = shop_data['title'] + ' ' + shop_data['descrirption']
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [16]:
shop_data.shape

(35548, 8)

In [17]:
vectorizer = CountVectorizer(ngram_range=(1, 2))

In [20]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
negative_texts = [' '.join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [' '.join(val) for val in shop_data['text'].values]
negative_texts[3]

'предел совершенство добиться понять brэхо целый 365 впереди'

In [21]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
negative_texts = [' '.join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [' '.join(val) for val in shop_data['text'].values]

In [22]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

**Разделяем на трейн и тест**

In [23]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels,
                                                    random_state=13)

In [24]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [25]:
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9803671415408605

In [26]:
with open('index_speaker.pkl', 'wb') as f:
    pickle.dump(index_map, f)

In [27]:
tfidf_vect = TfidfVectorizer().fit(X_train)
np.mean(tfidf_vect.idf_)

10.91980908257386

In [28]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

{'поговорить': 9.232794611287517,
 'разговор': 6.8409783845655285,
 'хороший': 10.149085343161671,
 'помошникнуть': 11.247697631829782,
 'тихо': 11.247697631829782,
 'спокойно': 11.247697631829782,
 'простить': 11.247697631829782,
 'сопля': 11.247697631829782,
 'босоножка': 11.247697631829782,
 'рр': 11.247697631829782,
 '37': 11.247697631829782,
 'новый': 11.247697631829782,
 'нравиться': 11.247697631829782,
 'старый': 11.247697631829782,
 'часть': 11.247697631829782,
 'обычный': 11.247697631829782,
 'спальная': 11.247697631829782,
 'район': 10.842232523721616,
 'левента': 11.247697631829782,
 'жаловать': 11.247697631829782,
 'толкуть': 11.247697631829782,
 'платье': 11.247697631829782,
 'большой': 11.247697631829782,
 'размер': 11.247697631829782,
 'нарядный': 11.247697631829782,
 'лёгкий': 11.247697631829782,
 'ткань': 11.247697631829782,
 'леопардовый': 11.247697631829782,
 'принт': 10.331406899955626,
 'открытый': 11.247697631829782,
 'плечо': 11.247697631829782,
 'подарокс': 11.2

In [29]:
ft_index_shop = annoy.AnnoyIndex(100, 'angular')
midf = np.mean(tfidf_vect.idf_)
index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, 'title'], shop_data.loc[i, 'image_links'])
    vector_ft = np.zeros(100)
    for word in shop_data.loc[i, 'text']:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * 1 # idfs.get(word, midf)
            n_ft += 1 # idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)
ft_index_shop.save('./data/shop.ann')

<ipython-input-29-5bf2734fe6c8>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(shop_data))):


  0%|          | 0/35548 [00:00<?, ?it/s]

True

In [30]:
ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('./data/shop.ann') 
midf = np.mean(tfidf_vect.idf_)

In [31]:
with open('index_shop.pkl', 'wb') as f:
    pickle.dump(index_map, f)

In [32]:
'бегать' in modelFT.wv

True

In [33]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * 1 # idfs.get(word, midf)
            n_ft += 1 # idfs.get(word, midf)
    return vector_ft / n_ft

In [34]:
ft_index_shop.get_nns_by_vector(np.zeros(100), 5, include_distances=True)

([29, 1253, 1615, 1915, 2509],
 [1.4142135381698608,
  1.4142135381698608,
  1.4142135381698608,
  1.4142135381698608,
  1.4142135381698608])

## Запускаем бота

In [37]:
updater = Updater(token, use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

In [38]:
# Определяем обработчики команд
def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день!')

def textMessage(update: Update, context: CallbackContext):
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([' '.join(input_txt)])
    prediction = lr.predict(vect)
    
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            update.message.reply_text('title: {} image: {}'.format(title, image))
            # bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.35:
        print(distances[0])
        update.message.reply_text('Не понимаю, повторите пожалуйста...')
        return
    update.message.reply_text(index_map[ft_index_val[0]])

In [40]:
# Запускаем команды в Telegram
dispatcher.add_handler(CommandHandler('start', startCommand))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage))

# Запускаем бота
updater.start_polling()
updater.idle()

2021-08-30 21:52:49,863 - apscheduler.scheduler - INFO - Scheduler started
2021-08-30 21:55:36,180 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-08-30 21:55:36,180 - apscheduler.scheduler - INFO - Scheduler has been shut down
